In [ ]:

import os
import sys
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
import pandas as pd
import numpy as np
import dask.dataframe as dd
import logging
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import mplfinance as mpf

# Asegurarse de que las gráficas se muestren en el notebook
%matplotlib inline

# Ajustar el valor de agg.path.chunksize para evitar el OverflowError
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

# Configuración de los logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Añadir la ruta base del proyecto a sys.path
base_path = os.path.abspath('..')
src_path = os.path.join(base_path, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Importar librerías
try:
    from data.data_loader import load_and_process_data
    from strategies.base_strategy import MovingAverageStrategy
    from utils.logger import Logger
    print("Importaciones realizadas correctamente")
except ModuleNotFoundError as e:
    print(f"Error en las importaciones: {e}")
    raise

def load_and_process_data(file_path):
    try:
        logger.info(f"Cargando y procesando datos desde {file_path} usando dask...")

        # Leer el archivo CSV usando dask
        ddf = dd.read_csv(
            file_path,
            header=0,  # Asumimos que el archivo CSV tiene encabezados
            names=["DateTime", "Bid", "Ask", "Volume"],
            dtype={"DateTime": str, "Bid": float, "Ask": float, "Volume": int}  # Especificar tipos de datos
        )

        # Convertir la columna 'DateTime' a tipo datetime
        ddf['DateTime'] = dd.to_datetime(ddf['DateTime'], format='%Y%m%d %H:%M:%S.%f', errors='coerce')

        # Filtrar filas con valores inválidos o NaN
        ddf = ddf.dropna()

        # Convertir el DataFrame de dask a pandas
        all_data = ddf.compute()

        # Configurar la columna 'DateTime' como índice
        all_data.set_index('DateTime', inplace=True)

        logger.info("Datos cargados y procesados correctamente.")
        return all_data

    except Exception as e:
        logger.error(f"Error al procesar los datos: {e}")
        return None

# Ruta al archivo CSV
data_path = '../src/data/data.csv'

# Cargar datos de mercado
try:
    logger = Logger(__name__)
    market_data = load_and_process_data(data_path)
    if market_data is not None:
        print("Datos de mercado cargados correctamente.")
        print(market_data.head())

        # Calcular el precio de cierre como el promedio de Bid y Ask
        market_data['close'] = (market_data['Bid'] + market_data['Ask']) / 2

        # Crear una instancia de la estrategia y obtener señales
        strategy = MovingAverageStrategy(short_window=40, long_window=100)
        signals = strategy.get_signal(market_data)

        # Visualizar solo una parte de los datos (por ejemplo, los últimos 1000 puntos)
        signals = signals.tail(1000)
        market_data = market_data.tail(1000)

        # Crear un DataFrame de ejemplo con datos OHLC
        ohlc_data = market_data[['Bid', 'Ask', 'Volume']].copy()
        ohlc_data['Open'] = ohlc_data['Bid']
        ohlc_data['High'] = ohlc_data[['Bid', 'Ask']].max(axis=1)
        ohlc_data['Low'] = ohlc_data[['Bid', 'Ask']].min(axis=1)
        ohlc_data['Close'] = ohlc_data['Ask']
        ohlc_data = ohlc_data[['Open', 'High', 'Low', 'Close', 'Volume']]

        # Identificar señales de compra y venta
        signals['signal'] = 0
        signals.loc[signals['short_mavg'] > signals['long_mavg'], 'signal'] = 1
        signals.loc[signals['short_mavg'] < signals['long_mavg'], 'signal'] = -1
        signals['buy_signal'] = signals['signal'].diff() == 2
        signals['sell_signal'] = signals['signal'].diff() == -2

        # Crear listas de anotaciones para las señales de compra y venta
        buy_signals = signals[signals['buy_signal']]
        sell_signals = signals[signals['sell_signal']]

        # Graficar velas japonesas con mplfinance y añadir señales de compra y venta
        apds = [
            mpf.make_addplot(signals['short_mavg'], color='cyan'),
            mpf.make_addplot(signals['long_mavg'], color='magenta'),
            mpf.make_addplot(buy_signals['close'], type='scatter', markersize=100, marker='^', color='green'),
            mpf.make_addplot(sell_signals['close'], type='scatter', markersize=100, marker='v', color='red')
        ]

        mpf.plot(
            ohlc_data,
            type='candle',
            style='charles',
            volume=True,
            title='EUR/USD Candlestick Chart with Buy/Sell Signals',
            ylabel='Price',
            ylabel_lower='Volume',
            figsize=(16, 8),
            tight_layout=True,
            addplot=apds
        )

        # Predicción de precios usando scikit-learn
        # Preparar los datos para la predicción
        X = market_data[['Bid', 'Ask', 'Volume']]
        y = market_data['close']

        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar un modelo de regresión lineal
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Hacer predicciones
        predictions = model.predict(X_test)

        # Visualizar las predicciones
        plt.figure(figsize=(16, 8))
        plt.plot(y_test.index, y_test, label='Actual', color='blue', linestyle='-', marker='o', markersize=5)
        plt.plot(y_test.index, predictions, label='Predicted', color='red', linestyle='--', marker='x', markersize=5)
        plt.title('Predicción de Precios de Cierre', fontsize=14, color='white')
        plt.xlabel('Fecha', fontsize=12, color='white')
        plt.ylabel('Precio', fontsize=12, color='white')
        plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.6)
        plt.legend(loc='upper left', fontsize=10)
        plt.xticks(rotation=45, fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout()
        plt.show()

    else:
        logger.error("No se pudieron cargar los datos")

except Exception as e:
    logger.error(f"Error al cargar los datos del mercado: {e}")